In [ ]:
# Importação de biblitecas
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
Adam

keras.optimizers.adam.Adam

In [ ]:
keras.optimizers.Adam

keras.optimizers.adam.Adam

In [ ]:
def load_data(url, dir_name):
  path_to_zip = tf.keras.utils.get_file(os.path.basename(url), origin=url, extract=True)

  dir = os.path.join(os.path.dirname(path_to_zip),dir_name)
  BATCH_SIZE = 32
  IMG_SIZE = (224, 224)

  dataset = tf.keras.utils.image_dataset_from_directory(
      dir,
      shuffle=True,
      batch_size=BATCH_SIZE,
      image_size=IMG_SIZE
  )
  return dataset

train_data = load_data('https://storage.googleapis.com/ibeans/train.zip', 'train')
validation_data = load_data('https://storage.googleapis.com/ibeans/validation.zip', 'validation')
test_data = load_data('https://storage.googleapis.com/ibeans/test.zip', 'test')

143812152/143812152 [==============================] - 3s 0us/step
Found 1034 files belonging to 3 classes.
18504213/18504213 [==============================] - 1s 0us/step
Found 133 files belonging to 3 classes.
17708541/17708541 [==============================] - 1s 0us/step
Found 128 files belonging to 3 classes.


In [ ]:
def freeze_layer(net,freeze_lay_name):

  # Getting layer number through layer name
  for layer_number, layer in enumerate(net.layers):
    if layer.name == freeze_lay_name:
      lay_nb = layer_number
      break

  # Froze layers
  for layer in base_model.layers[:lay_nb]:
    layer.trainable = False

  return base_model

In [ ]:
# Ajustamento do tamanho da imagem / Pré processamento
IMG_SIZE = 224

def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.
    return image, label

# Aplicando o redimensionamento nas bases
train_data = train_data.map(preprocess_image)
validation_data = validation_data.map(preprocess_image)
test_data = test_data.map(preprocess_image)

## 1) Treinamento do zero:

Use o modelo VGG16 sem treinamento prévio, remova a camada mais recente (camada de previsão) e adicione um classificador. Treine o modelo do zero e avalie.



In [ ]:
# Definição do modelo
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
x = global_average_layer(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# Compilação para definição de métricas
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinamento do modelo
history = model.fit(train_data, epochs=10, validation_data=validation_data)

Epoch 1/10
 5/33 [===>..........................] - ETA: 32:33 - loss: 10.1392 - accuracy: 0.4313

KeyboardInterrupt: ignored

In [ ]:
# Geração de métricas de perfomance
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)

# Print dos resultados
print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

4/4 [==============================] - 84s 20s/step - loss: 1.0969 - accuracy: 0.3516
Training accuracy: 0.3462282419204712
Training loss: 1.0970712900161743
Validation accuracy: 0.3308270573616028
Validation loss: 1.0969380140304565
Test accuracy: 0.3515625
Test loss: 1.096907138824463


## 2) Usando rede pré-treinada como extrator de recursos:

Use o mesmo modelo de Exp. 1, mas com o modelo VGG16 pré-treinado no ImageNet. Congele todas as camadas do ResNet50, treine e avalie o modelo.

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
base_model.trainable = False
x = global_average_layer(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# Treinamento
history = model.fit(train_data, epochs=10, validation_data=validation_data)

Epoch 1/10
33/33 [==============================] - 736s 22s/step - loss: 1.0471 - accuracy: 0.4584 - val_loss: 0.8385 - val_accuracy: 0.6917
Epoch 2/10
33/33 [==============================] - 713s 22s/step - loss: 0.7989 - accuracy: 0.6673 - val_loss: 0.7027 - val_accuracy: 0.7820
Epoch 3/10
33/33 [==============================] - 719s 22s/step - loss: 0.6979 - accuracy: 0.7137 - val_loss: 0.6699 - val_accuracy: 0.7444
Epoch 4/10
33/33 [==============================] - 716s 22s/step - loss: 0.6470 - accuracy: 0.7311 - val_loss: 0.5931 - val_accuracy: 0.7895
Epoch 5/10
33/33 [==============================] - 698s 21s/step - loss: 0.6273 - accuracy: 0.7408 - val_loss: 0.5760 - val_accuracy: 0.8045
Epoch 6/10
33/33 [==============================] - 704s 21s/step - loss: 0.5774 - accuracy: 0.7660 - val_loss: 0.5443 - val_accuracy: 0.8271
Epoch 7/10
33/33 [==============================] - 694s 21s/step - loss: 0.5446 - accuracy: 0.7766 - val_loss: 0.5217 - val_accuracy: 0.8195
Epoch 

In [ ]:
# Geração de métricas de perfomance
train_loss, train_acc = model.evaluate(train_features)
val_loss, val_acc = model.evaluate(val_features)
test_loss, test_acc = model.evaluate(test_features)

# Print dos resultados
print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

4/4 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Training accuracy: 0.0
Training loss: 0.0
Validation accuracy: 0.0
Validation loss: 0.0
Test accuracy: 0.0
Test loss: 0.0


In [ ]:
val_loss = model.evaluate(val_features)
test_loss = model.evaluate(test_features)


4/4 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


## 3-) Fine-tuning the latest layers: 

take a pre-trained VGG16 model, unfreeze the last blocks acording to the following:

### 3-a) - unfreeze the last convolutional blocks (from "block5_conv1"), train and evaluate the model



In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
base_model = freeze_layer(base_model,'block1_conv1')
base_model = freeze_layer(base_model,'block1_conv2')

base_model = freeze_layer(base_model,'block2_conv1')
base_model = freeze_layer(base_model,'block2_conv2')

base_model = freeze_layer(base_model,'block3_conv1')
base_model = freeze_layer(base_model,'block3_conv2')
base_model = freeze_layer(base_model,'block3_conv3')

base_model = freeze_layer(base_model,'block4_conv1')
base_model = freeze_layer(base_model,'block4_conv2')
base_model = freeze_layer(base_model,'block4_conv3')
x = global_average_layer(base_model.output)
#x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=validation_data)

In [ ]:
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)

print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

3-b) - unfreeze the last convolutional blocks (from "block4_conv1"), train and evaluate the model.

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
base_model = freeze_layer(base_model,'block1_conv1')
base_model = freeze_layer(base_model,'block1_conv2')

base_model = freeze_layer(base_model,'block2_conv1')
base_model = freeze_layer(base_model,'block2_conv2')

base_model = freeze_layer(base_model,'block3_conv1')
base_model = freeze_layer(base_model,'block3_conv2')
base_model = freeze_layer(base_model,'block3_conv3')

x = global_average_layer(base_model.output)
#x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=validation_data)

In [ ]:
# Mpetricas
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)

# Print resultados
print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

3-c) - unfreeze all convolutional blocks, train and evaluate the model.


In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

x = global_average_layer(base_model.output)
#x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=validation_data)

## Final results

Exp ||	train acc	|| train loss	|| val acc || val loss	|| test acc	|| test loss ||
--- || --- || --- || --- || --- || --- || --- ||
1 || 0.336 || 1.097 || 0.338 || 1.098 || 0.335 || 1.097 ||
2 || 0.790 || 0.541 || 0.812 || 0.583 || 0.757 || 0.566 ||
3-a) || 0.336 || 1.098 || 0.338 || 1.098 || 0.335 || 1.098 ||
3-b) || 0.333 || 1.098 || 0.330 || 1.098 || 0.335 || 1.098 ||
3-c) || 0.336 || 1.096 || 0.337 || 1.098 || 0.334 || 1.097 ||